In [113]:
rd_format = [24, 20, "Rd"]
rs_format = [19, 15, "Rs"]
rt_format = [14, 10, "Rt"]
opcode_range = [30, 25]
MSb_dict = {
    "position": 31,
    "single_format": "0",
    "bundle" : "1"
}

isa_dict = {
    "single_format": {
        "goto": {
            "opcode": "000001",
            "offset": [24, 4, "imm"],
            "cond":[3, 0, "0000"]
        },

        "br": {
            "opcode": "000001",
            "offset": [24, 4, "imm"],
            "cond":[3, 0, "cond"]
        },

        "stop": {
            "opcode": "001000",
            "reserved": [24, 0, "reserved"]
        },

        "test": {
            "opcode": "001100",
            "reserved0": [24, 20, "reserved"],
            "rs": True,
            "rt": True,
            "reserved1": [9, 0, "reserved"]
        },

        "cmp": {
            "opcode": "001101",
            "reserved0": [24, 20, "reserved"],
            "rs": True,
            "rt": True,
            "reserved1": [9, 0, "reserved"]
        },

        "fbr": {
            "opcode": "010100",
            "rd": True,
            "reserved": [19, 4, "reserved"],
            "cond": [3, 0, "cond"]
        },

        "fmr": {
            "opcode": "010101",
            "rd": True,
            "reserved": [19, 3, "reserved"],
            "cond": [2, 0, "q"]
        },

        "ldi": {
            "opcode": "010110",
            "rd": True,
            "reserved": [19, 0, "imm"]
        },

        "ldui": {
            "opcode": "010111",
            "rd": True,
            "rs": True,
            "reserved": [14, 0, "imm"]
        },

        "or": {
            "opcode": "011000",
            "rd": True,
            "rs": True,
            "rt": True,
            "reserved": [9, 0, "imm"]
        },

        "xor": {
            "opcode": "011001",
            "rd": True,
            "rs": True,
            "rt": True,
            "reserved": [9, 0, "imm"]
        },

        "and": {
            "opcode": "011010",
            "rd": True,
            "rs": True,
            "rt": True,
            "reserved": [9, 0, "imm"]
        },

        "not": {
            "opcode": "011011",
            "rd": True,
            "rt": True,
            "reserved0": [9, 0, "reserved"],
            "reserved1": [19, 15, "reserved"]
        },

        "addc": {
            "opcode": "011100",
            "rd": True,
            "rs": True,
            "rt": True,
            "reserved": [9, 0, "reserved"],
        },

        "subc": {
            "opcode": "011101",
            "rd": True,
            "rs": True,
            "rt": True,
            "reserved": [9, 0, "reserved"],
        },

        "add": {
            "opcode": "011110",
            "rd": True,
            "rs": True,
            "rt": True,
            "reserved": [9, 0, "reserved"],
        },

        "subc": {
            "opcode": "011111",
            "rd": True,
            "rs": True,
            "rt": True,
            "reserved": [9, 0, "reserved"],
        }
    }
}

In [114]:
def expand_fields(fields):
    new_fields = {}
    for key, item in fields.items():
        if key == "rd":
            new_fields["rd"] = rd_format
        elif key == "rs":
            new_fields["rs"] = rs_format
        elif key == "rt":
            new_fields["rt"] = rt_format
        elif key == "opcode":
            new_fields["opcode"] = (opcode_range[0], opcode_range[1], item)
        else:
            new_fields[key] = item

    return new_fields

In [115]:
def gen_row_content(name):
    found = False
    for key, item in isa_dict.items():
        if name in item:
            found = True
            fields = item[name]
            break

    if not found:
        raise ValueError("Given instruction ({}) not found".format(name))

    fields = expand_fields(fields)
    print(fields)

In [116]:
def check_completeness(insn_name, insn_fields):
    if not isinstance(insn_fields, list):
        raise ValueError("{}: Given insn_fields ({}) is not a list."
                         " \nFields are: {}".format(insn_name, type(insn_fields), insn_fields))
    expected_pos = 31
    for field in insn_fields:
        if field[1][0] != expected_pos:
            raise ValueError("{}: Given fields position ({}) does not match expected({})."
                             " \nFields are: {}".format(insn_name, field[1][0], expected_pos, insn_fields))
            
        if len(field[1]) == 2:
            expected_pos = expected_pos - 1
        else:
            if field[1][1] > expected_pos:
                raise ValueError("{}: Given fields position is incorrect:({}, {})."
                                  " \nFields are: {}".format(insn_name, field[1][0], field[1][1], insn_fields))
            expected_pos = field[1][1] - 1
            

In [136]:
def is_binary(num):
    try:
        int(num, 2)
        return True
    except ValueError:
        return False
    
def resolve_dict(isa_dict):
    new_dict = {}
    for format_key, format_insn_dict in isa_dict.items():
        for insn_name, compact_insn_fields in format_insn_dict.items():
            expanded_insn_fields = {}
            if format_key == "single_format":
                expanded_insn_fields["MSb"] = [MSb_dict["position"], MSb_dict["single_format"]]
            else:
                expanded_insn_fields["MSb"] = [MSb_dict["position"], MSb_dict["bundle"]]
                
            for field_name, compact_field_value in compact_insn_fields.items():
                if field_name == "opcode":
                    if not is_binary(compact_field_value):
                        raise ValueError("Given opcode is not binary({}).".format(compact_field_value))
                                         
                    if len(compact_field_value) != 6:
                        raise ValueError("Given opcode length ({}) does not match required ({}).".format(
                                len(compact_field_value), 6))
                        
                    expanded_insn_fields["opcode"] =[opcode_range[0], opcode_range[1], compact_field_value]
                
                elif field_name.lower() == "rd":
                    expanded_insn_fields["rd"] = rd_format
                elif field_name.lower() == "rs":
                    expanded_insn_fields["rs"] = rs_format
                elif field_name.lower() == "rt":
                    expanded_insn_fields["rt"] = rt_format
                else:
                    if (not isinstance(compact_field_value, list)) or (len(compact_field_value) != 3):
                        raise ValueError("Given field format not correct: ", compact_field_value)
                    if field_name.lower().startswith("reserve"):
                        compact_field_value[2] = r"\reserve"
                    if compact_field_value[2] == "imm":
                        compact_field_value[2] = "imm{}".format(compact_field_value[0] - compact_field_value[1] + 1)
                    expanded_insn_fields[field_name.lower()] = compact_field_value
            
            # sort the dict and save it as a tuple
#             print(expanded_insn_fields)
            keys = sorted(expanded_insn_fields, key = lambda x: expanded_insn_fields[x][0], reverse=True)
            sorted_insn_fields = [[key,expanded_insn_fields[key]] for key in keys]
            
            check_completeness(insn_name.lower(), sorted_insn_fields)
            
            new_dict[insn_name.lower()] = sorted_insn_fields
    
    
    return new_dict

In [139]:
resolved_isa_dict = resolve_dict(isa_dict)

In [142]:
fields = resolved_isa_dict["add"]
fields

[['MSb', [31, '0']],
 ['opcode', [30, 25, '011110']],
 ['rd', [24, 20, 'Rd']],
 ['rs', [19, 15, 'Rs']],
 ['rt', [14, 10, 'Rt']],
 ['reserved', [9, 0, '\\reserve']]]

In [152]:
fields

[['MSb', [31, '0']],
 ['opcode', [30, 25, '011110']],
 ['rd', [24, 20, 'Rd']],
 ['rs', [19, 15, 'Rs']],
 ['rt', [14, 10, 'Rt']],
 ['reserved', [9, 0, '\\reserve']]]

In [201]:
def gen_row_content(name):
    if not (name in resolved_isa_dict):
        raise ValueError("Given instruction ({}) not found".format(name))

    fields = resolved_isa_dict[name]

    row_string = ""
    font = "\\small"
    for field in fields:
        field_str = ""
        field_name = field[0]
        field_info = field[1]
        if len(field_info) == 2:
            field_length = 1
            field_pos_len = 1
        else:
            field_pos_len = 2
            field_length = field_info[0] - field_info[1] + 1

        field_value = field_info[field_pos_len]
        is_binary_str = is_binary(field_value)

        if is_binary_str:
            if len(field_value) != field_length:
                raise ValueError("field length ({}) does not match. "
                    "field: {}".format(field_length, field))

            # left vertical bar: only added when it is the first bit
            # right vertical bar: only added when the last bit is 0
            # "&" symbol: added for each bit, except the last bit is 0.
            # comment: added only for the first bit
            for i in range(len(field_value)):
                left_vertical_bar = " "
                right_vertical_bar = " "
                new_col_symbol = "& "
                comment = ""

                if i == 0:       # judge left vertical bar
                    left_vertical_bar = "|"
                    comment = "% {:>8s}: pos: {:>10s}, value: {}".format(
                                field_name,
                                str(field_info[0:field_pos_len]),
                                field_value)

                # judge right vertical bar
                if (i == len(field_value) - 1) and \
                    (len(field_value) - field_info[0] == 1):
                    right_vertical_bar = "|"

                # judge the new column symbol
                if (i == len(field_value) - 1) and \
                    (len(field_value) - field_info[0] == 1):
                    right_vertical_bar = " "

                field_str += "\multicolumn{ 1}{" + left_vertical_bar +\
                             "@{}c@{}" + right_vertical_bar + "}{" +\
                             font + field_value[i] + "} " + new_col_symbol +\
                             comment + "\n"

        else:
            right_vertical_bar = " "
            new_col_symbol = "& "

            # if this field reaches the end of the instruction
            if ((field_pos_len == 1 and field_info[0] == 0) or
                    (field_pos_len == 2 and field_info[1] == 0)):
                right_vertical_bar = "|"
                new_col_symbol = " "

            comment = "% {:>8s}: pos: {:>10s}, value: {}".format(
                        field_name,
                        str(field_info[0:field_pos_len]),
                        field_value)

            field_str += "\multicolumn{" + "{:2}".format(field_length) +\
                         "}{|@{}c@{}" + right_vertical_bar + "}{" + font +\
                         field_value + "} " + new_col_symbol + comment + "\n"

        row_string += field_str
    row_string += "\\\\ \n\cline{1-32}"

    return row_string

In [202]:
print(gen_row_content("add"))

\multicolumn{ 1}{|@{}c@{} }{\small0} & %      MSb: pos:       [31], value: 0
\multicolumn{ 1}{|@{}c@{} }{\small0} & %   opcode: pos:   [30, 25], value: 011110
\multicolumn{ 1}{ @{}c@{} }{\small1} & 
\multicolumn{ 1}{ @{}c@{} }{\small1} & 
\multicolumn{ 1}{ @{}c@{} }{\small1} & 
\multicolumn{ 1}{ @{}c@{} }{\small1} & 
\multicolumn{ 1}{ @{}c@{} }{\small0} & 
\multicolumn{ 5}{|@{}c@{} }{\smallRd} & %       rd: pos:   [24, 20], value: Rd
\multicolumn{ 5}{|@{}c@{} }{\smallRs} & %       rs: pos:   [19, 15], value: Rs
\multicolumn{ 5}{|@{}c@{} }{\smallRt} & %       rt: pos:   [14, 10], value: Rt
\multicolumn{10}{|@{}c@{}|}{\small\reserve}  % reserved: pos:     [9, 0], value: \reserve
\\ 
\cline{1-32}


In [180]:
str([10, 8])

'[10, 8]'

In [199]:
"{:2}".format(8)

' 8'